# PS2-6 Spam Classification

In [ ]:
import numpy as np
import src.util as util
import src.svm as svm

### (a) Data Preprocessing

In [2]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    return [word.lower() for word in message.split(' ')]
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message. 

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    word_count = {}
    for message in messages:
        words = set(get_words(message)) # use set to avoid counting duplicates in the same message
        for word in words:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1
    # print(word_count)
    word_dictionary = {}
    index = 0
    for word, count in word_count.items():
        if count >= 5:
            word_dictionary[word] = index
            index += 1
    return word_dictionary
    # *** END CODE HERE ***


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each 
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that 
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    num_messages = len(messages)
    num_words = len(word_dictionary)
    word_array = np.zeros((num_messages, num_words))

    for row_index, message in enumerate(messages):
        words = get_words(message)
        for word in words:
            if word in word_dictionary:
                col_index = word_dictionary[word]
                word_array[row_index, col_index] += 1

    return word_array
    # *** END CODE HERE ***

In [3]:
train_messages, train_labels = util.load_spam_dataset('./data/ds6_train.tsv')
val_messages, val_labels = util.load_spam_dataset('./data/ds6_val.tsv')
test_messages, test_labels = util.load_spam_dataset('./data/ds6_test.tsv')

In [4]:
dictionary = create_dictionary(train_messages)

util.write_json('./output/p06_dictionary', dictionary)

train_matrix = transform_text(train_messages, dictionary)

np.savetxt('./output/p06_sample_train_matrix', train_matrix[:100,:])

val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

### (b) Implement Naive Bayes Classifier

In [5]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    vocab_size = matrix.shape[1]
    spam = matrix[labels == 1, :]
    non_spam = matrix[labels == 0, :]

    # Prob. of words occurring in messages in each class can be modeled 
    # as a multinomial distribution Mult(vocab_size, phi_k_y*).
    # E.g., phi_k_y1 represents the prob. of word k occurring in spam messages.
    phi_k_y1 = (spam.sum(axis=0) + 1) / (spam.sum() + vocab_size)   # Laplace smoothing
    phi_k_y0 = (non_spam.sum(axis=0) + 1) / (non_spam.sum() + vocab_size)   # Laplace smoothing
    phi_y1 = np.mean(labels)

    return (phi_k_y1, phi_k_y0, phi_y1)
    # *** END CODE HERE ***

In [6]:
naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

In [ ]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    phi_k_y1, phi_k_y0, phi_y1 = model
    log_phi_k_y1 = np.log(phi_k_y1)
    log_phi_k_y0 = np.log(phi_k_y0)
    log_phi_y1 = np.log(phi_y1)
    log_phi_y0 = np.log(1 - phi_y1)

    # use dot product to measure distribution similarity
    # do not need to compute prob explicitly.
    score_y1 = np.sum(matrix * log_phi_k_y1, axis=1) + log_phi_y1
    score_y0 = np.sum(matrix * log_phi_k_y0, axis=1) + log_phi_y0
    return score_y1 > score_y0
    # *** END CODE HERE ***

In [29]:
naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)
np.savetxt('./output/p06_naive_bayes_predictions', naive_bayes_predictions)
naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)
print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

Naive Bayes had an accuracy of 0.978494623655914 on the testing set


### (c) Check Indicative Words

In [ ]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    phi_k_y1, phi_k_y0, _ = model
    score = np.log(phi_k_y1 / phi_k_y0)
    reversed_dictionary = {v: k for k, v in dictionary.items()}

    top_five_idx = np.argsort(-score)[:5]   # in descending order
    return [reversed_dictionary[idx] for idx in top_five_idx]
    # *** END CODE HERE ***

In [35]:
top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)
print('The top 5 indicative words for Naive Bayes are: ', top_5_words)
util.write_json('./output/p06_top_indicative_words', top_5_words)

The top 5 indicative words for Naive Bayes are:  ['claim', 'won', 'prize', 'tone', 'urgent!']


### (d) Find Best RBF kernel for SVM

In [38]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    best_acc = -1
    best_radius = None
    for radius in radius_to_consider:
        svm_pred = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius)
        acc = (svm_pred == val_labels).mean()
        if acc > best_acc:
            best_acc = acc
            best_radius = radius
    return best_radius
    # *** END CODE HERE ***

In [39]:
optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])
util.write_json('./output/p06_optimal_radius', optimal_radius)
print('The optimal SVM radius was {}'.format(optimal_radius))
svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)
svm_accuracy = np.mean(svm_predictions == test_labels)
print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))

The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9695340501792115 on the testing set
